In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

### Функции

In [2]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()


def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [3]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999

На вебинаре мы использовали товар 999999 - что это за товар?  
    
    Искуственный товар который присвоен всем пользователям не вошедших в "топ".

Зачем он нужен?  
    
    Для того что бы не обрезать выборку

Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
    
        Ухудшаем качество за счет добавления большого количества одинакого товара у множества юзеров не покупавших товары из топ 5000

In [5]:
data_train_webinar = pd.read_csv('./data_train.csv')
data_test_webinar = pd.read_csv('./data_test.csv')

In [6]:
data_train_webinar.loc[data_train_webinar['item_id']== 999999]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
6,2375,26984851516,1,999999,1,1.57,364,-0.68,1642,1,0.0,0.0
8,2375,26984851516,1,999999,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
10,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2278476,2088,41297771158,635,999999,1,15.65,304,-5.25,1258,91,0.0,0.0
2278479,1541,41297771177,635,999999,1,1.05,304,0.00,1300,91,0.0,0.0
2278480,1541,41297771180,635,999999,1,5.19,304,0.00,1301,91,0.0,0.0
2278481,1168,41297772063,635,999999,1,12.40,304,0.00,1526,91,0.0,0.0


Можно ли удалить этот товар?   
    
    Можно но появяться новые пользователи(которые не покупали товары из топ 5000)

Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

In [7]:
data_train_webinar_drop999999 = data_train_webinar.drop(data_train_webinar[data_train_webinar.item_id == 999999].index)

In [8]:

new_test_users = len(set(data_test_webinar['user_id']) - set(data_train_webinar_drop999999['user_id']))

print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 5 новых юзеров


In [9]:
data_train_webinar.loc[data_train_webinar['item_id']== 999999]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
6,2375,26984851516,1,999999,1,1.57,364,-0.68,1642,1,0.0,0.0
8,2375,26984851516,1,999999,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
10,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2278476,2088,41297771158,635,999999,1,15.65,304,-5.25,1258,91,0.0,0.0
2278479,1541,41297771177,635,999999,1,1.05,304,0.00,1300,91,0.0,0.0
2278480,1541,41297771180,635,999999,1,5.19,304,0.00,1301,91,0.0,0.0
2278481,1168,41297772063,635,999999,1,12.40,304,0.00,1526,91,0.0,0.0


In [10]:
data_train_webinar_drop999999.loc[data_train_webinar['item_id']== 999999]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc


In [11]:
result_top5000 = data_test_webinar.groupby('user_id')['item_id'].unique().reset_index()
result_top5000.columns=['user_id', 'actual']
result_top5000['actual'] = result_top5000['actual'].apply(lambda x: list(x))
result_top5000.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [12]:
%%time

items = data_train_webinar_drop999999.item_id.unique()

result_top5000['random_recommendation_top5000'] = result_top5000['user_id'].apply(lambda x: random_recommendation(items, n=5))
result_top5000.head(2)

CPU times: user 314 ms, sys: 4.08 ms, total: 318 ms
Wall time: 316 ms


,user_id,actual,random_recommendation_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[883004, 9296847, 926206, 5563693, 9526892]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[970714, 1013149, 9419525, 12188582, 1034134]"


In [13]:
result_top5000.apply(
    lambda x: precision_at_k(x['random_recommendation_top5000'], x['actual'],  5), axis=1
    ).mean()

0.006072477962781582

In [14]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train_webinar_drop999999, n=5)

result_top5000['popular_recommendation_top5000'] = result_top5000['user_id'].apply(lambda x: popular_recs)
result_top5000.head(2)

CPU times: user 52.7 ms, sys: 0 ns, total: 52.7 ms
Wall time: 53.8 ms


,user_id,actual,random_recommendation_top5000,popular_recommendation_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[883004, 9296847, 926206, 5563693, 9526892]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[970714, 1013149, 9419525, 12188582, 1034134]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [15]:
result_top5000.apply(
    lambda x: precision_at_k(x['popular_recommendation_top5000'], x['actual'],  5), axis=1
    ).mean()

0.15523996082272082

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [16]:
items_weights_1 = data.groupby('item_id')['sales_value'].sum().reset_index().rename({'sales_value': 'weight'}, axis=1) 
const_weights = 2    # что бы избавиться от 0 
items_weights_1['weight'] = np.log(items_weights_1['weight'] + const_weights)
items_weights_1['weight'] = items_weights_1['weight']/items_weights_1['weight'].sum()

In [17]:
items_weights_1.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000006
min,2.567100e+04,0.000003
25%,9.665830e+05,0.000007
50%,1.448516e+06,0.000010
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000050


In [18]:
items_weights_1.sum()

item_id    4.555646e+11
weight     1.000000e+00
dtype: float64

In [19]:
items_weights_2 = data.groupby('item_id')['sales_value'].sum().reset_index().rename({'sales_value': 'weight'}, axis=1) 

items_weights_2['weight'] = items_weights_2['weight'] + const_weights
items_weights_2['weight'] = items_weights_2['weight']/items_weights_2['weight'].sum()

In [20]:
items_weights_2.sum()

item_id    4.555646e+11
weight     1.000000e+00
dtype: float64

In [21]:
items_weights_3 = data.groupby('item_id')['sales_value'].sum().reset_index().rename({'sales_value': 'weight'}, axis=1) 

items_weights_3['weight'] = const_weights - 1/np.e**(items_weights_3['weight'])
items_weights_3['weight'] = items_weights_3['weight']/items_weights_3['weight'].sum()

In [22]:
items_weights_3.sum()

item_id    4.555646e+11
weight     1.000000e+00
dtype: float64

In [23]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights['item_id'], p=items_weights['weight'],  size=n, replace=False)
    
    
    return recs.tolist()

Сделайте предсказания

In [24]:
result1 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result1.columns=['user_id', 'actual']
result1['actual'] = result1['actual'].apply(lambda x: list(x))
result1.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [25]:
%%time
result1['weighted_random_recommendation'] = result1['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_1))
result1.head(10)
# your_code

CPU times: user 2.56 s, sys: 1.64 ms, total: 2.56 s
Wall time: 2.57 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[10456408, 10182734, 891337, 920371, 55201]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[954210, 13073042, 1775642, 1078069, 13877306]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1038437, 941067, 8351541, 6464074, 1125804]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1065238, 838282, 9485371, 14020092, 898824]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[12988067, 57570, 832815, 12456471, 983359]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[960452, 1099163, 1016800, 835482, 5126325]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[1039506, 1047964, 10255417, 1993094, 12386154]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[1062152, 9676877, 907817, 17383227, 819559]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[15741011, 873577, 895166, 954077, 1087753]"
9,16,"[1062973, 1082185, 13007710]","[1073224, 1142063, 79154, 962606, 874780]"


In [26]:
result2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result2['actual'] = result2['actual'].apply(lambda x: list(x))

In [27]:
%%time
result2['weighted_random_recommendation'] = result2['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_2))
result2.head(10)

CPU times: user 2.58 s, sys: 923 µs, total: 2.59 s
Wall time: 2.58 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1002771, 14025258, 1070702, 5564901, 6961484]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1038985, 871337, 8249140, 9575251, 822814]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1106523, 1104604, 871611, 1087102, 838797]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[883404, 1045779, 965766, 6633167, 828910]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[968687, 1116376, 6533765, 927191, 1404121]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[936594, 6534178, 15595960, 962845, 13210377]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[6633210, 1053690, 12754661, 866871, 12691988]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[957339, 893501, 923559, 1103808, 9575321]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[12262778, 6391137, 9527542, 948651, 9419888]"
9,16,"[1062973, 1082185, 13007710]","[1106523, 893018, 9446386, 5571176, 1075007]"


In [28]:
result3 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result3.columns=['user_id', 'actual']
result3['actual'] = result3['actual'].apply(lambda x: list(x))

In [29]:
%%time
result3['weighted_random_recommendation'] = result3['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_3))
result3.head(10)

CPU times: user 2.55 s, sys: 740 µs, total: 2.55 s
Wall time: 2.55 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12731302, 2386213, 1029402, 43411, 902856]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1085138, 42832, 3938236, 1258099, 854646]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[9659557, 13910409, 7137147, 872915, 1067606]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6701389, 889055, 10284862, 1155362, 9446336]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15831117, 1060719, 10457323, 1007044, 15716911]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[9682163, 10459759, 29674, 2787407, 1017235]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[916374, 16809581, 5572944, 819890, 712280]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[1125290, 5569531, 2776524, 6534515, 1002556]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[898312, 838752, 10462111, 1109371, 1048816]"
9,16,"[1062973, 1082185, 13007710]","[945800, 9337120, 6391517, 1076684, 47792]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. 

In [30]:
result = pd.read_csv('preds.csv')# загрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1124548, 49213, 1291147, 858892, 1036347]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1037722, 1003073, 12673030, 943442, 1097373]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [31]:
type(result['actual'][0])

str

In [32]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [33]:
random_recommendation_result = result.apply(
    lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1
    ).mean()

popular_recommendation_result = result.apply(
    lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
itemitem_result = result.apply(
    lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1
    ).mean()
    
cosine_result = result.apply(
    lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1
    ).mean()
    
tfidf_result = result.apply(
    lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1
    ).mean()
    
own_purchases_result = result.apply(
    lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1
    ).mean()

    
weighted_random_recommendation_result1 = result1.apply(
    lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
weighted_random_recommendation_result2 = result2.apply(
    lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
weighted_random_recommendation_result3 = result3.apply(
    lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual'],  5), axis=1
    ).mean()
    
result_recom_dict = {
    'random_recommendation_result':random_recommendation_result,
    'weighted_random_recommendation_result1': weighted_random_recommendation_result1,
    'weighted_random_recommendation_result2': weighted_random_recommendation_result2,
    'weighted_random_recommendation_result3': weighted_random_recommendation_result3,
    'popular_recommendation_result': popular_recommendation_result,
    'itemitem_result(k=5)': itemitem_result,
    'cosine_result': cosine_result,
    'tfidf_result': tfidf_result,
    'own_purchases_result': own_purchases_result,
    }

In [34]:
for key, value in result_recom_dict.items(): print(f'{key} \t\t {value}') 

random_recommendation_result 		 0.0004897159647404506
weighted_random_recommendation_result1 		 0.0011753183153770813
weighted_random_recommendation_result2 		 0.025073457394711146
weighted_random_recommendation_result3 		 0.0006856023506366307
popular_recommendation_result 		 0.15523996082272082
itemitem_result(k=5) 		 0.13692458374142857
cosine_result 		 0.13290891283055686
tfidf_result 		 0.1389813907933383
own_purchases_result 		 0.17969311132876015


Какой алгоритм показывает лучшее качество?

    own_purchases(трюк с itemitem для ближашего 1-ого соседа)
    
Почему?
    
    рекомендация на основе наипохожего товара

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [35]:
%%time

user_item_matrix = pd.pivot_table(data_train_webinar, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

CPU times: user 3.22 s, sys: 575 ms, total: 3.8 s
Wall time: 3.82 s


In [36]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [43]:
for k in range(1, 6):
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=False, 
                        recalculate_user=True)
    result['itemitem@'+str(k)] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,itemitem@1,itemitem@2,itemitem@3,itemitem@4,itemitem@5
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1124548, 49213, 1291147, 858892, 1036347]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[999999, 1082185, 1029743, 995785, 1004906]","[999999, 1082185, 995242, 1029743, 840361]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 995242, 1127831]","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1037722, 1003073, 12673030, 943442, 1097373]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[999999, 1082185, 1098066, 6534178, 1127831]","[999999, 1082185, 1098066, 6534178, 826249]","[999999, 1082185, 981760, 1098066, 6534178]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[897181, 1057168, 59305, 1513242, 1060987]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[999999, 1082185, 1029743, 6534178, 1127831]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 1127831, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[6395992, 1136794, 1788928, 1616802, 13525069]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]","[999999, 1082185, 1029743, 1127831, 995785]","[999999, 1082185, 995242, 1029743, 826249]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 1127831, 995242]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1211927, 1160773, 12132685, 1044992, 937759]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]","[999999, 1082185, 1029743, 1098066, 6534178]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 6534178, 995242]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 1127831, 1098066]"


In [44]:
for k in range(1, 6):
    itemitem_result = result.apply(
        lambda x: precision_at_k(x['itemitem@'+str(k)], x['actual'],  5), axis=1
        ).mean()
    print(f'K={k}\tprecision@{k} - {itemitem_result}')
    

K=1	precision@1 - 0.17969311132876015
K=2	precision@2 - 0.1920094678419821
K=3	precision@3 - 0.1860920666013684
K=4	precision@4 - 0.14495592556317158
K=5	precision@5 - 0.13692458374142857


### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.